In [1]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision

In [2]:
EPOCH = 1
BATCH_SIZE = 50
LR = 0.001
DOWNLOAD_MNIST = False

In [3]:
train_data = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=torchvision.transforms.ToTensor(), download=DOWNLOAD_MNIST,)
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

test_data = torchvision.datasets.MNIST(root='./mnist/', train=False)
test_x = torch.unsqueeze(test_data.test_data, dim=1).type(torch.FloatTensor)[:2000]/255.   # Tensor on GPU
test_y = test_data.test_labels[:2000]

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2,),
                                   nn.ReLU(), nn.MaxPool2d(kernel_size=2),)
        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, 5, 1, 2), nn.ReLU(), nn.MaxPool2d(2),)
        self.out = nn.Linear(32 * 7 * 7, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output

In [5]:
cnn = CNN()
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()

In [7]:
for epoch in range(EPOCH):
    for step, (x, y) in enumerate(train_loader):

        # !!!!!!!! Change in here !!!!!!!!! #
        b_x = x   # Tensor on GPU
        b_y = y    # Tensor on GPU

        output = cnn(b_x)
        loss = loss_func(output, b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % 50 == 0:
            test_output = cnn(test_x)

            # !!!!!!!! Change in here !!!!!!!!! #
            pred_y = torch.max(test_output, 1)[1].data  # move the computation in GPU

            accuracy = torch.sum(pred_y == test_y).type(torch.FloatTensor) / test_y.size(0)
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)

Epoch:  0 | train loss: 2.3057 | test accuracy: 0.23
Epoch:  0 | train loss: 0.4274 | test accuracy: 0.80
Epoch:  0 | train loss: 0.3252 | test accuracy: 0.90
Epoch:  0 | train loss: 0.2084 | test accuracy: 0.92
Epoch:  0 | train loss: 0.2312 | test accuracy: 0.93
Epoch:  0 | train loss: 0.1775 | test accuracy: 0.95
Epoch:  0 | train loss: 0.1457 | test accuracy: 0.95
Epoch:  0 | train loss: 0.0894 | test accuracy: 0.96
Epoch:  0 | train loss: 0.4636 | test accuracy: 0.96
Epoch:  0 | train loss: 0.0724 | test accuracy: 0.96
Epoch:  0 | train loss: 0.1807 | test accuracy: 0.97
Epoch:  0 | train loss: 0.0350 | test accuracy: 0.97
Epoch:  0 | train loss: 0.1711 | test accuracy: 0.97
Epoch:  0 | train loss: 0.0844 | test accuracy: 0.97
Epoch:  0 | train loss: 0.0305 | test accuracy: 0.98
Epoch:  0 | train loss: 0.0858 | test accuracy: 0.96
Epoch:  0 | train loss: 0.0325 | test accuracy: 0.98
Epoch:  0 | train loss: 0.0085 | test accuracy: 0.98
Epoch:  0 | train loss: 0.0684 | test accuracy

In [8]:
test_output = cnn(test_x[:10])
pred_y = torch.max(test_output, 1)[1].data # move the computation in GPU

print(pred_y, 'prediction number')
print(test_y[:10], 'real number')

tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9]) prediction number
tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9]) real number
